In [ ]:
import requests
import pandas as pd

api_url = "https://ows.goszakup.gov.kz/v3/refs/ref_units"
token = "d5c3d78fc111d88a0a37b4ab8f83cbd5"
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}
fetch_params = {"limit": 1000} 
file = "mkey_units.csv"


def retrieve_mkey_units():
    """Получает данные МКЕЙ через API и формирует список."""
    mkei_list = []
    next_query = None

    while True:
        try:
            response = requests.get(api_url, headers=headers, params=fetch_params, timeout=10)
            response.raise_for_status()
            response_data = response.json()

            records = response_data.get("items", [])
            if not records:
                print("Все данные загружены.")
                break

            for record in records:
                mkei_list.append({
                    "ID": record.get("code"),
                    "name_kz": record.get("name_kz"),
                    "name_ru": record.get("name_ru"),
                    "code": record.get("code"),
                    "code2": record.get("code2"),
                    "alpha_code": record.get("alpha_code")
                })

            print(f"Загружено записей: {len(mkei_list)}")
            export_to_csv(mkei_list)  

            next_query = response_data.get("next_page")
            if not next_query:
                break

        except requests.exceptions.RequestException as err:
            print(f"Ошибка запроса: {err}")
            break

    return mkey_list


def export_to_csv(dataset):
    """Записывает данные в CSV-файл."""
    if not dataset:
        print("Нет данных для экспорта.")
        return

    data_frame = pd.DataFrame(dataset)
    data_frame.to_csv(
        file,
        mode='a', 
        header=not pd.io.common.file_exists(file),  
        index=False,
        encoding="utf-8-sig",
        sep="|",
        quotechar="'",
        escapechar="\\"
    )
    print(f"Данные сохранены в {file}")


retrieve_mkey_units()